Импорты

In [1]:
import json
import pandas as pd
from transliterate  import translit
from yargy.tokenizer import MorphTokenizer
import pymorphy2
from yargy import (
    Parser,
    rule, or_, and_, not_,
)

from yargy.predicates import (
    caseless, type, gram, normalized,
    in_, in_caseless, dictionary
)

from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp
INT = type('INT')

/Users/user/.local/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Загрузка файлов

In [2]:
bread = pd.read_json('bread-probe.json')
milk = pd.read_json('milk-normal.json')

Функция получения производителей

In [36]:
def getVendors(data):
    vendorlist = {}
    for i in range(0,len(data['manufacturer'])):
        item = data['manufacturer'][i]['name'];
        if(item not in vendorlist):
            vendorlist[item] = [item]
            alt = translit(item, 'ru');
            if(alt != item):
                vendorlist[item].append(alt)
    return vendorlist

Функция поиска вхождений слов

In [4]:
def findWordOccurance(data):
    occ = {}
    for name in data['name'].values:
        words = name.split()
        for word in words:
            word = word.upper().strip()
            if not word in occ:
                occ[word] = 0
            occ[word] = occ[word]+1
    sorted_dict = {}
    sorted_keys = sorted(occ, key=occ.get,reverse=True)

    for key in sorted_keys:
        sorted_dict[key] = occ[key]
    return sorted_dict

Нормализация слов

In [37]:
def normalizeKeys(orgKeys,vendorlist=[]):
    morph = pymorphy2.MorphAnalyzer()
    tokenizer = MorphTokenizer()
    for key in list(orgKeys):
        keyData = list(tokenizer(key))
        p = morph.parse(key)[0]
    
        if(key in vendorlist):
            orgKeys.remove(key)
            continue
        if(len(key) <= 1):
            orgKeys.remove(key)
            continue
        if(len(keyData)>1):
            orgKeys.remove(key)
            continue
        if({'UNKN'} in p.tag):
            orgKeys.remove(key)
            continue
        if({'NUMB'} in p.tag):
            orgKeys.remove(key)
            continue
    return orgKeys

Перевод слова в транслит

In [6]:
def translitNames(names):
    for name in names:
        alt = translit(name, 'ru');
        if(alt != name):
            names.append(alt)
    return names

**Дальше алгоритм**

*Определяем категории*

In [38]:
def getCategoriesAndStuff(data):
    vendorlist = getVendors(data)
    words = findWordOccurance(data)
    keys = list((words.keys()))
    keysNormalized = normalizeKeys(keys,vendorlist)
    categoryName = list(keysNormalized)[0]
    keysNormalized.remove(categoryName)
    categoryID = data["category_id"].unique().tolist()
    return {"name":categoryName,"keywords":keysNormalized,"vendorlist":vendorlist,"categoryID":categoryID}

In [39]:
milkCategory = getCategoriesAndStuff(milk)
breadCategory = getCategoriesAndStuff(bread)

Непосредстваенно алгоритм поиска

In [40]:
allCategories = {}
allCategories[milkCategory['name']] = milkCategory
allCategories[breadCategory['name']] = breadCategory

In [41]:
allCategories

{'МОЛОКО': {'name': 'МОЛОКО',
  'keywords': ['БЗМЖ',
   'УЛЬТРАПАСТЕРИЗОВАННОЕ',
   'ПАСТЕРИЗОВАННОЕ',
   'ДОМИК',
   'ДЕРЕВНЕ',
   'СТЕРИЛИЗОВАННОЕ',
   'МЛ',
   'БОЛЬШАЯ',
   'КРУЖКА',
   'БЕЛЫЙ',
   'ГОРОД',
   'ПРАВИЛЬНОЕ',
   'БЕЗЛАКТОЗНОЕ',
   'ОТБОРНОЕ',
   'МОЛОЧНИК',
   'ТОПЛЕНОЕ',
   'ПИТЬЕВОЕ',
   'ВЕСЕЛЫЙ',
   'FINE',
   'LIFE',
   'КОЗЬЕ',
   'ЦЕЛЬНОЕ',
   'ДЕТСКОЕ',
   'АСЕНЬЕВСКАЯ',
   'ФЕРМА',
   'BARISTA',
   'КОНЦЕНТРАТ',
   'МЕСЯЦЕВ',
   'УГЛЕЧЕ',
   'ПОЛЕ',
   'ORGANIC',
   'MOLOKO',
   'EILA',
   'СТЕРИЗОВАННОЕ',
   'БЕЗ',
   'ЗАМЕНИТЕЛЯ',
   'МОЛОЧНОГО',
   'DIETALAT',
   'КАРАМЕЛЬ',
   'НИЗКОЛАКТОЗНОЕ',
   'ЛЕСНОЙ',
   'ОРЕХ',
   'ELITE',
   'ДЛЯ',
   'ДЕТЕЙ',
   'PROF',
   'LINE',
   'БИОЛАКТ',
   'ОБОГАЩЕННОЕ',
   'СВЕЖЕЕ',
   'ЗАВТРА',
   'ПИСКАРЕВСКИЙ',
   'ОСОБОЕ',
   'КЛЕВЕРОК',
   'ВЫСШЕГО',
   'КАЧЕСТВА',
   'ФАСОЛЬ',
   'СЕЛО',
   'ЗЕЛЕНОЕ',
   'ВЕСЁЛЫЙ',
   'ХОЗЯЙСТВО',
   'ВАСИЛЬЕВОЙ',
   'АВ',
   'МАСЛО',
   'GROUP',
   'ДИЕТАЛАТ',
   'ШТ',
   'ОБЕЗЖИ

Этап 1.Определение категории

In [42]:
text = ''' 3 буханки Еврохлеба,  хлеба Гербер  три бутылки молока Простоквашино, 9 бутылок молока Простоквашино'''

In [43]:
Item = fact(
    'item',
    ['amount','vendor', 'category','unit','type']
)

In [44]:
list(allCategories.keys())

['МОЛОКО', 'ХЛЕБ']

In [45]:
TYPE = morph_pipeline(list(allCategories.keys()))
ITEM_Category = rule(
    TYPE.interpretation(
        Item.type
    )
).interpretation(
    Item
)

In [46]:
parser = Parser(ITEM_Category)
match = parser.find(text)
for match in parser.findall(text):
    category = [_.value for _ in match.tokens][0]

In [47]:
category

'молока'

Этап 2. Определение производителя

In [48]:
morph = pymorphy2.MorphAnalyzer()
categoryName = morph.parse(category)[0].normal_form.upper()
categoryName

'МОЛОКО'

In [89]:
def getVendorList(data):
    vendorList = []
    vendorValues = list(data.values())
    for alt in vendorValues:
        for value in alt:
            vendorList.append(value)
    return vendorList

In [96]:
def findVendorData(vendorData,vendor):
    for vendorName in vendorData.keys():
        if(vendor.upper() in vendorData[vendorName]):
            return vendorName
    return ""

In [98]:
findVendorData(allCategories[categoryName]["vendorlist"],"ВАЛИо")

'VALIO'

In [90]:
getVendorList(allCategories[categoryName]["vendorlist"])

['VALIO',
 'ВАЛИО',
 'RIOBA',
 'РИОБА',
 'ARO',
 'АРО',
 'PARMALAT',
 'ПАРМАЛАТ',
 'G-BALANCE',
 'Г-БАЛАНЦЕ',
 'ДОМИК В ДЕРЕВНЕ',
 'ПРОСТОКВАШИНО',
 'БЕЛЫЙ ГОРОД',
 'ПЕТМОЛ',
 'ТЕМА',
 'ЭКОНИВА',
 'АСЕНЬЕВСКАЯ ФЕРМА',
 'ПИСКАРЕВСКОЕ',
 'ФРУТОНЯНЯ',
 'PROMILKER',
 'ПРОМИЛКЕР',
 'СВЕЖЕЕ ЗАВТРА',
 'КЛЕВЕР',
 'БОЛЬШАЯ КРУЖКА',
 'ВЕСЕЛЫЙ МОЛОЧНИК',
 'ВКУСНОТЕЕВО',
 'УГЛЕЧЕ ПОЛЕ',
 'FASOL',
 'ФАСОЛ',
 'FINE LIFE',
 'ФИНЕ ЛИФЕ',
 'АИСФЕР',
 'БРЯНСКИЙ МОЛОЧНЫЙ КОМБИНАТ',
 'СЕЛО ЗЕЛЕНОЕ',
 'BETTAMILK',
 'БЕТТАМИЛК',
 'ХОЗЯЙСТВО ВАСИЛЬЕВОЙ АВ',
 'MOLOKO GROUP',
 'МОЛОКО ГРОУП']

In [52]:
VENDOR = morph_pipeline(allCategories[categoryName]["vendorlist"].keys())

In [53]:
ITEM_Vendor = rule(
    VENDOR.interpretation(
        Item.vendor
    )
).interpretation(
    Item
)

In [54]:
parser = Parser(ITEM_Vendor)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    vendor = [_.value for _ in match.tokens][0]

Match(
    tokens=[MorphToken(
         value='Простоквашино',
         span=[56, 69),
         type='RU',
         forms=[Form('простоквашин', Grams(ADJF,Poss,neut,nomn,sing)),
          Form('простоквашин', Grams(ADJF,Poss,accs,neut,sing)),
          Form('простоквашино', Grams(Geox,NOUN,Sgtm,inan,neut,nomn,sing)),
          Form('простоквашино', Grams(Geox,NOUN,Sgtm,accs,inan,neut,sing))]
     )],
    span=[56, 69)
)

Этап 3 определение кол-ва и прочего

In [24]:
categoryName

'МОЛОКО'

In [25]:
CATEGORY = morph_pipeline([categoryName])

In [26]:
TYPE

PipelineRule(
    pipeline=MorphPipeline(
        keys=[Key(
             value='МОЛОКО',
             terms=[{'молоко'}]
         ),
         Key(
             value='ХЛЕБ',
             terms=[{'хлеб'}]
         )]
    )
)

In [27]:
AMOUNT =  INT.interpretation(
    interp.custom(int)
)

In [28]:
UNIT = morph_pipeline(["бутылка"])

In [29]:
LITERALS = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
}

In [30]:
AMOUNT = dictionary(LITERALS).interpretation(
    interp.normalized().custom(LITERALS.get)
)

AMOUNT_NUM =  INT.interpretation(
    interp.custom(int)
)



In [32]:
 ITEM_MISC = rule(
        AMOUNT.interpretation(
            Item.amount
        ).optional(),
        AMOUNT_NUM.interpretation(
            Item.amount
        ).optional(),
        UNIT.interpretation(
            Item.unit
        ).optional(),
        CATEGORY.interpretation(
            Item.category
        ),
        VENDOR.interpretation(
            Item.vendor
        ).optional()
    ).interpretation(
        Item
    )
   

In [33]:
parser = Parser(ITEM_MISC)
match = parser.find(text)
display(match)
for match in parser.findall(text):
    vendor = [_.value for _ in match.tokens][0]

Match(
    tokens=[MorphToken(
         value='три',
         span=[37, 40),
         type='RU',
         forms=[Form('три', Grams(NUMR,accs,inan)),
          Form('три', Grams(NUMR,nomn)),
          Form('тереть', Grams(VERB,excl,impf,impr,sing,tran))]
     ),
     MorphToken(
         value='бутылки',
         span=[41, 48),
         type='RU',
         forms=[Form('бутылка', Grams(NOUN,femn,gent,inan,sing)),
          Form('бутылка', Grams(NOUN,accs,femn,inan,plur)),
          Form('бутылка', Grams(NOUN,femn,inan,nomn,plur))]
     ),
     MorphToken(
         value='молока',
         span=[49, 55),
         type='RU',
         forms=[Form('молоко', Grams(NOUN,Sgtm,gent,inan,neut,sing)),
          Form('молока', Grams(NOUN,femn,inan,nomn,sing))]
     ),
     MorphToken(
         value='Простоквашино',
         span=[56, 69),
         type='RU',
         forms=[Form('простоквашин', Grams(ADJF,Poss,neut,nomn,sing)),
          Form('простоквашин', Grams(ADJF,Poss,accs,neut,sing)),
     

In [34]:
for match in parser.findall(text):
    itemsData = []
    for itemData in [_.value for _ in match.tokens]:
        itemsData.append(itemData)
    print(itemsData)

['три', 'бутылки', 'молока', 'Простоквашино']
['9', 'бутылок', 'молока', 'Простоквашино']


In [ ]:
def findVendor

In [ ]:
os.environ['w2n.lang'] = 'ru'

w2n.word_to_num('пятьдесят')


In [ ]:
match.fact.vendor

In [ ]:
match.fact.type

In [ ]:
match.fact.amount

Заключительный этап. Поиск по БД

In [ ]:
milk['manufacturer']

In [ ]:
milk[milk['manufacturer'].item()['name'] == "VALIO"]

In [ ]:
milk[milk['manufacturer'].name.upper() ==  match.fact.vendor]